In [ ]:
import re

def nomor_perkara(text):
    # Cari pola nomor perkara, tetap fleksibel terhadap spasi
    match = re.search(
        r'\d+\s*/\s*pid\.sus-tpk\s*/\s*\d+\s*/\s*pn\s+sby',
        text,
        flags=re.IGNORECASE
    )

    if match:
        return match.group()  # Jangan hapus spasi, biar tetap 'pn sby'
    else:
        return None


In [ ]:
import re

def amar_putusan(text):
    # Normalisasi angka dan spasi
    text = re.sub(r"(\d+)\.(\S)", r"\1. \2", text)
    text = re.sub(r";\s*(\d+)\.", r"; \1.", text)

    # Toleransi spasi antar huruf 'mengadili'
    pattern_mengadili = re.compile(r"m\s*e\s*n\s*g\s*a\s*d\s*i\s*l\s*i", flags=re.IGNORECASE)
    matches = list(pattern_mengadili.finditer(text))
    if not matches:
        return ""

    last_match = matches[-1]
    amar_text = text[last_match.end():].strip()

    # Ambil isi dari 1. sampai sebelum ; 5.
    match = re.search(r"1\.\s*(.*?)(?=;\s*5\.)", amar_text, flags=re.DOTALL)
    if not match:
        return ""

    combined_text = match.group(1)

    # Pisahkan jadi 4 poin berdasarkan ; 2., ; 3., ; 4.
    split_points = re.split(r";\s*2\.\s*|\s*;\s*3\.\s*|\s*;\s*4\.\s*", combined_text)

    # Ambil maksimal 4 poin dan bersihkan
    cleaned = []
    for p in split_points[:4]:
        sentence = re.sub(r'\s+', ' ', p.strip())
        # Tambahkan titik jika belum ada di akhir
        if not sentence.endswith('.'):
            sentence += '.'
        cleaned.append(sentence)

    # Gabungkan dengan transisi
    if len(cleaned) == 4:
        return f"{cleaned[0]} Selain itu, {cleaned[1]} Selanjutnya, {cleaned[2]} Terakhir, {cleaned[3]}"
    else:
        return ' '.join(cleaned)


In [ ]:
import re

def extract_barang_bukti(text):
    # Deteksi kata 'mengadili' dengan toleransi spasi
    pattern_mengadili = re.compile(r"m\s*e\s*n\s*g\s*a\s*d\s*i\s*l\s*i\s*:?", flags=re.IGNORECASE)
    matches = list(pattern_mengadili.finditer(text))
    if not matches:
        return ""

    # Ambil teks setelah 'mengadili'
    last_match = matches[-1]
    amar_text = text[last_match.end():].strip()

    # Cari frasa 'barang bukti berupa'
    bb_header = re.search(
        r"(menetapkan\s+)?barang\s+bukti\s+berupa\s*:?", amar_text, flags=re.IGNORECASE
    )
    if not bb_header:
        return ""

    # Ambil teks setelah bagian header barang bukti
    bb_text = amar_text[bb_header.end():].strip()

    # Potong sebelum poin selanjutnya yang diawali angka + kata kerja hukum
    stop_match = re.search(
        r"\d+\.\s*(menetapkan|membebankan|memerintahkan|memulihkan|menyatakan)",
        bb_text, flags=re.IGNORECASE | re.MULTILINE
    )
    if stop_match:
        bb_text = bb_text[:stop_match.start()].strip()

    # Ambil poin-poin barang bukti
    items = re.findall(r"(?:\d+[\.\)])\s*(.*?)(?=\n\s*\d+[\.\)]|\Z)", bb_text, flags=re.DOTALL)

    if not items:
        return ""

    # Bersihkan
    cleaned = [re.sub(r'\s+', ' ', item.strip()) for item in items]
    joined = ' dan '.join(cleaned)

    return f"Barang bukti berupa {joined}." if joined else ""

In [ ]:
import re

def extract_pekerjaan(text):
    # 1. Bagi berdasarkan terdakwa: pakai romawi diikuti angka dan titik (misalnya: i.1., ii.1., dst)
    # Asumsikan bagian terdakwa diakhiri sebelum "mengadili" atau "menimbang" atau teks utama dimulai
    parts = re.split(r"(?=\b(?:i{1,3}|iv|v|vi|vii|viii|ix|x)[.]\s*1[.])", text, flags=re.IGNORECASE)

    pekerjaan_list = []

    for part in parts:
        # 2. Cari 'pekerjaan' di setiap bagian
        match = re.search(
            r"(?i)pekerjaan\s*:?\s*(.+?)(?=\s*(?:agama|pendidikan|terdakwa|status|[0-9]{1,2}[.)])\b|[;\n\.]|$)",
            part
        )
        if match:
            pekerjaan = re.sub(r'\s+', ' ', re.sub(r',+', '', match.group(1))).strip()
            pekerjaan_list.append(pekerjaan)

    return pekerjaan_list


In [ ]:
import re

def extract_penuntut_umum(text):
    # Tangani jika HANYA ada Penuntut Umum TANPA nama
    pattern_tanpa_nama = r"(?:dengan|serta)[\s]*dihadiri[\s]*oleh[\s]*(?:Jaksa[\s]*Penuntut[\s]*Umum|Penuntut[\s]*Umum|JaksaPenuntutUmum|PenuntutUmum)(?![\w])"
    if re.search(pattern_tanpa_nama, text, re.IGNORECASE | re.DOTALL):
        return "tidak disebutkan"

    # Tangani jika nama disebutkan sebelum frasa penuntut umum
    pattern_nama = r"(?:dengan|serta)[\s]*dihadiri[\s]*oleh[\s]*([\w\s.,()-]+?)\s*(?:selaku|sebagai)?\s*(?:Jaksa[\s]*Penuntut[\s]*Umum|Penuntut[\s]*Umum|JaksaPenuntutUmum|PenuntutUmum)"
    match = re.search(pattern_nama, text, re.IGNORECASE | re.DOTALL)
    if match:
        return match.group(1).strip()

    return "tidak ada informasi"


In [ ]:
import re

def nama_terdakwa(text):
    pattern = r"(?i)(?:\d\.\s*)?(?:nama lengkap|nama)\s*:?\s*([\s\S]{3,100}?)(?=\s*(?:\d?[.)]?\s*)?tempat\s+lahir)"
    return [m.strip() for m in re.findall(pattern, text)]

In [ ]:
import re

def extract_metadata(text):
    data = {}

    # Nomor perkara
    data['nomor_perkara'] = nomor_perkara(text)

    # m = re.findall(r"nama lengkap\s*:?\s*([a-zA-Z.,\-\s]+)", text, flags=re.IGNORECASE)
    data['nama_terdakwa'] = nama_terdakwa(text)

    # Pekerjaan
    data['pekerjaan'] = extract_pekerjaan(text)

    # perkara
    m = re.search(r"tindak pidana\s*:?\s*([a-z\s\-\,]+)", text)
    data['tindak pidana'] = m.group(1).strip().split()[0] if m else ""

    # Pasal
    m = re.search(r"Pasal\s+[^\n]*?Tahun\s+\d{4}", text, flags=re.IGNORECASE)
    data['pasal'] = m.group(0).strip() if m else ""

    # penuntut umum / penggugat
    data['penuntut_umum'] = extract_penuntut_umum(text)

    # Tanggal (optional)
    m = re.search(r"(diputuskan|putusan)[^\n]*tanggal\s+(\d{1,2}\s+[A-Za-z]+\s+\d{4})", text, flags=re.IGNORECASE)
    data['tanggal_putusan'] = m.group(2).strip() if m else ""

    # penuntut umum / penggugat
    data['amar_putusan'] = amar_putusan(text)

    # penuntut umum / penggugat
    data['ringkasan_barang_bukti'] = extract_barang_bukti(text)

    return data

In [ ]:
import pandas as pd
import os

# Misal nama file yang mau diuji
filename = "data/raw/case_030.txt"

# Baca file
with open(os.path.join(filename), 'r', encoding='utf-8') as f:
    text = f.read()
    metadata = extract_metadata(text)
    metadata['case_id'] = "case_001"
    metadata['text_full'] = text

# Simpan ke DataFrame (1 baris saja)
df = pd.DataFrame([metadata])
df

,nomor_perkara,nama_terdakwa,pekerjaan,tindak pidana,pasal,penuntut_umum,tanggal_putusan,amar_putusan,ringkasan_barang_bukti,case_id,text_full
0,111/pid.sus-tpk/2024/pn sby,[aniningsih binti riyadi],[wiraswasta],korupsi,pasal 2 ayat (1) jo. pasal 18 uu r.i. nomor 31...,tidak disebutkan,24 desember 2024,menyatakan terdakwa aniningsih binti riyadi te...,Barang bukti berupa 1 (satu) bundel asli data ...,case_001,putusan nomor 111/pid.sus-tpk/2024/pn sby dem...


In [ ]:
import pandas as pd
import os

# Folder input
input_dir = "data/raw"

# List untuk menampung metadata semua file
all_data = []

# Loop semua file .txt
for filename in os.listdir(input_dir):
    if filename.endswith(".txt"):
        file_path = os.path.join(input_dir, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
            metadata = extract_metadata(text)
            metadata['case_id'] = os.path.splitext(filename)[0]
            metadata['text_full'] = text
            all_data.append(metadata)

# Buat DataFrame dari semua metadata
df = pd.DataFrame(all_data)
# Hitung length (jumlah kata)
df["length_text_full"] = df["text_full"].astype(str).apply(lambda x: len(x.split()))

# Tampilkan DataFrame
print(df)


                  nomor_perkara  \
0   140/pid.sus-tpk/2024/pn sby   
1    86/pid.sus-tpk/2024/pn sby   
2     3/pid.sus-tpk/2025/pn sby   
3     9/pid.sus-tpk/2025/pn sby   
4    42/pid.sus-tpk/2024/pn sby   
5   140/pid.sus-tpk/2024/pn sby   
6    42/pid.sus-tpk/2024/pn sby   
7    65/pid.sus-tpk/2024/pn sby   
8   107/pid.sus-tpk/2024/pn sby   
9   136/pid.sus-tpk/2024/pn sby   
10  109/pid.sus-tpk/2024/pn sby   
11   92/pid.sus-tpk/2024/pn sby   
12    8/pid.sus-tpk/2025/pn sby   
13   83/pid.sus-tpk/2024/pn sby   
14   12/pid.sus-tpk/2025/pn sby   
15   14/pid.sus-tpk/2025/pn sby   
16   82/pid.sus-tpk/2024/pn sby   
17   72/pid.sus-tpk/2024/pn sby   
18   99/pid.sus-tpk/2024/pn sby   
19   65/pid.sus-tpk/2024/pn sby   
20   15/pid.sus-tpk/2025/pn sby   
21  113/pid.sus-tpk/2024/pn sby   
22   87/pid.sus-tpk/2024/pn sby   
23    2/pid.sus-tpk/2025/pn sby   
24   26/pid.sus-tpk/2025/pn sby   
25   90/pid.sus-tpk/2024/pn sby   
26  111/pid.sus-tpk/2024/pn sby   
27   71/pid.sus-tpk/

In [ ]:
df.to_json("data/processed/cases.json", orient='records', indent=2, force_ascii=False)